# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Часть 5: составление списков приставок и лемм
1. Собрать список приставок
2. Вытащить из базы глаголы с нужными приставками
3. Отфильтровать по частотности
4. Убрать мусор и пересечения (ручная разметка)
5. Сохранить в датафрейм

#### Импорт модулей

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import re
import pickle
import pymongo
import pandas as pd
from tqdm.auto import tqdm

**Подключение к базе**

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client['thesis']
# fs = gridfs.GridFS(db)
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'thesis')

In [4]:
sentences = db.sentences
lemmas = db.lemmas
tokens = db.tokens

#### Список приставок
С опорой на (Кронгауз 1998)

![](krongauz_prefixes.jpg)

In [5]:
prefixes = ['бе[зс]', 'во?', 'в[зс]', 'вне', 'во[зс]', 'все', 'вы', 'до', 'за', 'и[зс]', 'на', 'над', 'наи', 'не', 'недо', 'ни', 'ни[зс]', 'о', 'обе[зс]', 'от', 'па', 'пере', 'по', 'под', 'поза', 'после', 'пра', 'пре', 'пред', 'при', 'про', 'противо', 'ра[зс]', 'ро[зс]', 'с', 'су', 'тре', 'у']

1:80: E501 line too long (294 > 79 characters)


In [6]:
len(prefixes)

38

#### Поиск глаголов

In [7]:
def find_verbs_with_prefix(prefix_name):
    regexp = '^{}'.format(prefix_name)
    found = lemmas.find({'lemma': {'$regex': regexp}, 'pos': 'V'},
                        {'lemma': True, 'freq': True, 'freq_0': True,
                         'freq_1': True, 'freq_2': True, '_id': False}
                        ).sort('freq', -1)
    return list(found)

In [8]:
prefix_dict = {}
for prefix in tqdm(prefixes):
    prefix_dict[prefix] = find_verbs_with_prefix(prefix)

  0%|          | 0/38 [00:00<?, ?it/s]

In [9]:
len(prefix_dict['по'])  # :))

10262

#### Фильтрация по частотности

In [10]:
# тут мб фильтр по частотности / еще каким-то параметрам
# и в принципе посмотреть на частотность по разным периодам

# не меньше 100 раз в период

Сразу убираем слова с очень маленькой частотностью (<5 в период), т.к. они даже не попадут в векторные модели.

In [11]:
prefix_dict_5 = {}

for prefix in prefix_dict:
    verbs_5 = []
    for verb in prefix_dict[prefix]:
        if verb['freq_0'] >= 5 and verb['freq_1'] >= 5 and verb['freq_2'] >= 5:
            verbs_5.append(verb)
    if verbs_5:
        prefix_dict_5[prefix] = verbs_5
    else:
        print(f'No verbs found for prefix {prefix}.')

In [12]:
# сортировка для удобства чтения
prefix_dict_5 = dict(sorted(list(prefix_dict_5.items())))

for prefix in prefix_dict_5:
    prefix_dict_5[prefix] = sorted(prefix_dict_5[prefix],
                                   key=lambda x: x['lemma'])

In [13]:
prefixes_5 = list(prefix_dict_5.keys())
len(prefixes_5)

38

In [14]:
for prefix in prefix_dict_5:
    print(prefix, len(prefix_dict_5[prefix]))

бе[зс] 15
в[зс] 177
вне 2
во? 501
во[зс] 137
все 3
вы 449
до 173
за 954
и[зс] 287
на 549
над 45
наи 3
не 91
недо 28
ни 16
ни[зс] 11
о 1310
обе[зс] 33
от 432
па 26
пере 362
по 1361
под 316
поза 11
после 2
пра 8
пре 135
пред 62
при 421
про 512
противо 9
ра[зс] 634
ро[зс] 5
с 961
су 27
тре 24
у 502


#### Ручная очистка
Поскольку наш метод извлечения глаголов с приставками далек от совершенства, выделим из полученных списков глаголы, которые туда попали справедливо.

In [15]:
filtered_prefs = {}

In [16]:
def edit_prefix(prefix, prefix_dict):
    filtered_verbs = []
    for verb in prefix_dict[prefix]:
        keep = input(f'{verb["lemma"]} -- приставка {prefix}? +/-/? ')
        verb_copy = verb
        verb_copy['keep'] = keep
        filtered_verbs.append(verb_copy)
    return filtered_verbs

In [17]:
for i, prefix in enumerate(list(prefix_dict_5.keys())):
    print()
    print(f'----- Приставка {i}: {prefix} -----')
    filtered_prefs[prefix] = edit_prefix(prefix, prefix_dict_5)
    last_modified = i
    print()
    filter_next = input('Продолжить фильтрацию? да / нет ')
    if filter_next.lower() == 'нет':
        print(f'Последняя измененная приставка -- {prefix} (индекс {last_modified}).')
        break


----- Приставка 0: бе[зс] -----
бездействовать -- приставка бе[зс]? +/-/? +
бездельничать -- приставка бе[зс]? +/-/? +
безмолвствовать -- приставка бе[зс]? +/-/? +
безобразить -- приставка бе[зс]? +/-/? +
безобразничать -- приставка бе[зс]? +/-/? +
безуметь -- приставка бе[зс]? +/-/? +
безумствовать -- приставка бе[зс]? +/-/? +
беседовать -- приставка бе[зс]? +/-/? -
бесить -- приставка бе[зс]? +/-/? -
беситься -- приставка бе[зс]? +/-/? -
бесноваться -- приставка бе[зс]? +/-/? -
беспокоить -- приставка бе[зс]? +/-/? +
беспокоиться -- приставка бе[зс]? +/-/? +
бесчестить -- приставка бе[зс]? +/-/? +
бесчинствовать -- приставка бе[зс]? +/-/? +

Продолжить фильтрацию? да / нет да

----- Приставка 1: в[зс] -----
взаимодействовать -- приставка в[зс]? +/-/? -
взалкать -- приставка в[зс]? +/-/? +
взбадривать -- приставка в[зс]? +/-/? +
взбадриваться -- приставка в[зс]? +/-/? +
взбаламутить -- приставка в[зс]? +/-/? +
взбалтывать -- приставка в[зс]? +/-/? +
взбегать -- приставка в[зс]? +/-/?

варьировать -- приставка во?? +/-/? -
варьироваться -- приставка во?? +/-/? -
вашбродить -- приставка во?? +/-/? -
ваять -- приставка во?? +/-/? -
вбегать -- приставка во?? +/-/? +
вбивать -- приставка во?? +/-/? +
вбиваться -- приставка во?? +/-/? +
вбирать -- приставка во?? +/-/? +
вбрасывать -- приставка во?? +/-/? +
вваливать -- приставка во?? +/-/? +
вваливаться -- приставка во?? +/-/? +
ввергать -- приставка во?? +/-/? +
ввергаться -- приставка во?? +/-/? +
ввертывать -- приставка во?? +/-/? +
вверять -- приставка во?? +/-/? +
вверяться -- приставка во?? +/-/? +
ввестись -- приставка во?? +/-/? +
ввинчивать -- приставка во?? +/-/? +
ввинчиваться -- приставка во?? +/-/? +
вводить -- приставка во?? +/-/? +
вводиться -- приставка во?? +/-/? +
ввозить -- приставка во?? +/-/? +
ввозиться -- приставка во?? +/-/? +
ввязывать -- приставка во?? +/-/? +
ввязываться -- приставка во?? +/-/? +
вгибать -- приставка во?? +/-/? +
вглядываться -- приставка во?? +/-/? +
вгонять -- приставка во?? +

вклеивать -- приставка во?? +/-/? +
вклиниваться -- приставка во?? +/-/? +
включать -- приставка во?? +/-/? +
включаться -- приставка во?? +/-/? +
вколачивать -- приставка во?? +/-/? +
вколачиваться -- приставка во?? +/-/? +
вкопать -- приставка во?? +/-/? +
вкоренять -- приставка во?? +/-/? +
вкрадываться -- приставка во?? +/-/? +
вкрапливать -- приставка во?? +/-/? +
вкушать -- приставка во?? +/-/? +
влагать -- приставка во?? +/-/? +
владеть -- приставка во?? +/-/? -
владычествовать -- приставка во?? +/-/? -
вламываться -- приставка во?? +/-/? +
властвовать -- приставка во?? +/-/? -
влачить -- приставка во?? +/-/? -
влачиться -- приставка во?? +/-/? -
влезать -- приставка во?? +/-/? +
влеплять -- приставка во?? +/-/? +
влепляться -- приставка во?? +/-/? +
влетать -- приставка во?? +/-/? +
влечь -- приставка во?? +/-/? -
влечься -- приставка во?? +/-/? -
вливать -- приставка во?? +/-/? +
вливаться -- приставка во?? +/-/? +
влипать -- приставка во?? +/-/? +
влиять -- приставка во?? +/-

восторгаться -- приставка во?? +/-/? -
восторжествовать -- приставка во?? +/-/? -
востребовать -- приставка во?? +/-/? -
вострепетать -- приставка во?? +/-/? -
вострить -- приставка во?? +/-/? -
вострубить -- приставка во?? +/-/? -
восхвалять -- приставка во?? +/-/? -
восхваляться -- приставка во?? +/-/? -
восхищать -- приставка во?? +/-/? -
восхищаться -- приставка во?? +/-/? -
восходить -- приставка во?? +/-/? -
восчувствовать -- приставка во?? +/-/? -
воцаряться -- приставка во?? +/-/? +
вочеловечиваться -- приставка во?? +/-/? +
впадать -- приставка во?? +/-/? +
впаивать -- приставка во?? +/-/? +
впархивать -- приставка во?? +/-/? +
вперять -- приставка во?? +/-/? +
вперяться -- приставка во?? +/-/? +
впечатлевать -- приставка во?? +/-/? +
впечатлить -- приставка во?? +/-/? +
впечатлять -- приставка во?? +/-/? +
впивать -- приставка во?? +/-/? +
впиваться -- приставка во?? +/-/? +
впираться -- приставка во?? +/-/? +
вписывать -- приставка во?? +/-/? +
вписываться -- приставка во?? 

вывалить -- приставка вы? +/-/? +
вывалиться -- приставка вы? +/-/? +
вывалять -- приставка вы? +/-/? +
вываляться -- приставка вы? +/-/? +
вываривать -- приставка вы? +/-/? +
вывариваться -- приставка вы? +/-/? +
выведывать -- приставка вы? +/-/? +
вывертываться -- приставка вы? +/-/? +
выверять -- приставка вы? +/-/? +
вывесить -- приставка вы? +/-/? +
вывести -- приставка вы? +/-/? +
вывесть -- приставка вы? +/-/? +
выветривать -- приставка вы? +/-/? +
выветриваться -- приставка вы? +/-/? +
вывешивать -- приставка вы? +/-/? +
вывешиваться -- приставка вы? +/-/? +
вывинчивать -- приставка вы? +/-/? +
вывихивать -- приставка вы? +/-/? +
выводить -- приставка вы? +/-/? +
выводиться -- приставка вы? +/-/? +
вывозить -- приставка вы? +/-/? +
вывозиться -- приставка вы? +/-/? +
выволакивать -- приставка вы? +/-/? +
выворачивать -- приставка вы? +/-/? +
выворачиваться -- приставка вы? +/-/? +
вывязывать -- приставка вы? +/-/? +
выгадывать -- приставка вы? +/-/? +
выгибать -- приставка вы? 

...

состариться -- приставка с? +/-/? +
состоять -- приставка с? +/-/? +
состояться -- приставка с? +/-/? +
сострадать -- приставка с? +/-/? +
сострить -- приставка с? +/-/? +
состроить -- приставка с? +/-/? +
состряпать -- приставка с? +/-/? +
состязаться -- приставка с? +/-/? +
сосуществовать -- приставка с? +/-/? +
сосчитать -- приставка с? +/-/? +
сосчитывать -- приставка с? +/-/? +
сотворить -- приставка с? +/-/? +
сотвориться -- приставка с? +/-/? +
сотворять -- приставка с? +/-/? +
соткать -- приставка с? +/-/? +
соткаться -- приставка с? +/-/? +
сотрудничать -- приставка с? +/-/? +
сотрясать -- приставка с? +/-/? +
сотрясаться -- приставка с? +/-/? +
соучаствовать -- приставка с? +/-/? +
софокать -- приставка с? +/-/? -
сохнуть -- приставка с? +/-/? -
сохранять -- приставка с? +/-/? +
сохраняться -- приставка с? +/-/? +
сочетать -- приставка с? +/-/? +
сочетаться -- приставка с? +/-/? +
сочиниться -- приставка с? +/-/? +
сочинять -- приставка с? +/-/? +
сочиняться -- приставка с? +

стремиться -- приставка с? +/-/? -
стреноживать -- приставка с? +/-/? +
стрескать -- приставка с? +/-/? +
стричь -- приставка с? +/-/? -
стричься -- приставка с? +/-/? -
строгать -- приставка с? +/-/? +
строить -- приставка с? +/-/? -
строиться -- приставка с? +/-/? -
строчить -- приставка с? +/-/? -
стругать -- приставка с? +/-/? -
струить -- приставка с? +/-/? -
струиться -- приставка с? +/-/? -
струсить -- приставка с? +/-/? +
струхнуть -- приставка с? +/-/? +
струшивать -- приставка с? +/-/? +
стряпать -- приставка с? +/-/? -
стрясать -- приставка с? +/-/? +
стрясаться -- приставка с? +/-/? +
стряхивать -- приставка с? +/-/? +
стряхиваться -- приставка с? +/-/? +
студить -- приставка с? +/-/? -
стукать -- приставка с? +/-/? -
стукаться -- приставка с? +/-/? -
стукнуть -- приставка с? +/-/? -
стукнуться -- приставка с? +/-/? -
ступать -- приставка с? +/-/? -
стухнуть -- приставка с? +/-/? +
стучать -- приставка с? +/-/? -
стучаться -- приставка с? +/-/? -
стушевываться -- приставка 

угадываться -- приставка у? +/-/? +
угасать -- приставка у? +/-/? +
угашать -- приставка у? +/-/? +
углублять -- приставка у? +/-/? +
углубляться -- приставка у? +/-/? +
углядеть -- приставка у? +/-/? +
угнать -- приставка у? +/-/? +
угнаться -- приставка у? +/-/? +
угнездиться -- приставка у? +/-/? +
угнетать -- приставка у? +/-/? +
угнетаться -- приставка у? +/-/? +
уговаривать -- приставка у? +/-/? +
уговариваться -- приставка у? +/-/? +
угодничать -- приставка у? +/-/? +
угождать -- приставка у? +/-/? +
угомонять -- приставка у? +/-/? +
угомоняться -- приставка у? +/-/? +
угонять -- приставка у? +/-/? +
угоняться -- приставка у? +/-/? +
угораздить -- приставка у? +/-/? +
угорать -- приставка у? +/-/? +
уготавливать -- приставка у? +/-/? +
уготовать -- приставка у? +/-/? +
уготовлять -- приставка у? +/-/? +
угощать -- приставка у? +/-/? +
угощаться -- приставка у? +/-/? +
угреваться -- приставка у? +/-/? +
угрожать -- приставка у? +/-/? +
угрызать -- приставка у? +/-/? +
угрызаться 

употреблять -- приставка у? +/-/? +
употребляться -- приставка у? +/-/? +
управить -- приставка у? +/-/? +
управлять -- приставка у? +/-/? +
управляться -- приставка у? +/-/? +
упражнять -- приставка у? +/-/? +
упражняться -- приставка у? +/-/? +
упразднять -- приставка у? +/-/? +
упраздняться -- приставка у? +/-/? +
упрашивать -- приставка у? +/-/? +
упревать -- приставка у? +/-/? +
упреждать -- приставка у? +/-/? +
упрекать -- приставка у? +/-/? +
упрочивать -- приставка у? +/-/? +
упрочиваться -- приставка у? +/-/? +
упрощать -- приставка у? +/-/? +
упрощаться -- приставка у? +/-/? +
упрямиться -- приставка у? +/-/? +
упрятывать -- приставка у? +/-/? +
упускать -- приставка у? +/-/? +
упускаться -- приставка у? +/-/? +
уравнивать -- приставка у? +/-/? +
уравниваться -- приставка у? +/-/? +
уравновешивать -- приставка у? +/-/? +
уравновешиваться -- приставка у? +/-/? +
уразумевать -- приставка у? +/-/? +
урегулировать -- приставка у? +/-/? +
урезать -- приставка у? +/-/? +
урезониват

9:80: E501 line too long (86 > 79 characters)


Добавить полу-

In [26]:
prefix_dict_5['полу'] = list(
    lemmas.find({'lemma': {'$regex': '^полу'}, 'pos': 'V',
                 'freq_0': {'$gt': 4}, 'freq_1': {'$gt': 4},
                 'freq_2': {'$gt': 4}},
                {'lemma': True, 'freq': True, 'freq_0': True,
                 'freq_1': True, 'freq_2': True, '_id': False}
                ).sort('lemma'))

len(prefix_dict_5['полу'])

19

In [27]:
filtered_prefs['полу'] = edit_prefix('полу', prefix_dict_5)

полузадушить -- приставка полу? +/-/? +
полузакрывать -- приставка полу? +/-/? +
полузамерзать -- приставка полу? +/-/? +
полузасыпать -- приставка полу? +/-/? +
полуистлевать -- приставка полу? +/-/? +
полулежать -- приставка полу? +/-/? +
полуобвалиться -- приставка полу? +/-/? +
полуобертываться -- приставка полу? +/-/? +
полуоткрыть -- приставка полу? +/-/? +
полуотрывать -- приставка полу? +/-/? +
полуприкрывать -- приставка полу? +/-/? +
полуразваливаться -- приставка полу? +/-/? +
полураздавливать -- приставка полу? +/-/? +
полурасстегнуть -- приставка полу? +/-/? +
полусгнивать -- приставка полу? +/-/? +
полусидеть -- приставка полу? +/-/? +
полустереть -- приставка полу? +/-/? +
получать -- приставка полу? +/-/? -
получаться -- приставка полу? +/-/? -


In [28]:
filtered_prefs = dict(sorted(list(filtered_prefs.items())))

In [29]:
with open('prefixes_filtered.pickle', 'wb') as f_raw:
    pickle.dump(filtered_prefs, f_raw)

In [30]:
pandas_rows = []
for prefix in filtered_prefs:
    for verb in filtered_prefs[prefix]:
        for_pandas = verb
        for_pandas['prefix'] = prefix
        pandas_rows.append(for_pandas)

In [31]:
pref_df = pd.DataFrame(pandas_rows)
cols = pref_df.columns.tolist()
cols

['lemma', 'freq', 'freq_0', 'freq_1', 'freq_2', 'keep', 'prefix']

In [32]:
# меняем порядок колонок
cols = ['prefix', 'lemma', 'freq', 'freq_0', 'freq_1', 'freq_2', 'keep']
pref_df = pref_df[cols]

# оставляем только нужные строки и столбцы
pref_df = pref_df.loc[pref_df['keep'] == '+']
pref_df.reset_index(inplace=True)
pref_df.drop(columns=['index', 'keep'], inplace=True)

In [3]:
pref_df

,prefix,lemma,freq,freq_0,freq_1,freq_2
0,бе[зс],бездействовать,541,155,255,131
1,бе[зс],бездельничать,308,41,169,98
2,бе[зс],безмолвствовать,726,361,191,174
3,бе[зс],безобразить,149,99,43,7
4,бе[зс],безобразничать,306,110,136,60
...,...,...,...,...,...,...
8454,у,ущемляться,62,8,16,38
8455,у,ущипывать,510,147,222,141
8456,у,уязвлять,1305,606,411,288
8457,у,уяснять,1803,677,740,386


#### Статистика по приставкам

In [4]:
pref_df.groupby(['prefix']).size()

prefix
бе[зс]       11
в[зс]       150
во?         137
во[зс]      122
все           1
вы          444
до          153
за          937
и[зс]       276
на          524
над          17
не            4
недо         17
ни[зс]       10
о           817
обе[зс]      27
от          393
пере        357
по         1009
под         266
полу         17
пре          68
пред         58
при         411
про         483
противо       9
ра[зс]      622
с           653
у           466
dtype: int64

Порог для включения приставки в исследование -- 10 глаголов (т.е. нужно как минимум 10 разных глаголов с одной приставкой, чтобы можно было судить о семантических сдвигах в ее значении). Отфильтруем приставки, для которых не набралось нужного количества лемм.

In [5]:
final_pref_df = pref_df.groupby(['prefix']).filter(lambda x: len(x) >= 10)
final_pref_df

,prefix,lemma,freq,freq_0,freq_1,freq_2
0,бе[зс],бездействовать,541,155,255,131
1,бе[зс],бездельничать,308,41,169,98
2,бе[зс],безмолвствовать,726,361,191,174
3,бе[зс],безобразить,149,99,43,7
4,бе[зс],безобразничать,306,110,136,60
...,...,...,...,...,...,...
8454,у,ущемляться,62,8,16,38
8455,у,ущипывать,510,147,222,141
8456,у,уязвлять,1305,606,411,288
8457,у,уяснять,1803,677,740,386


In [6]:
final_pref_df.groupby(['prefix']).size()

prefix
бе[зс]       11
в[зс]       150
во?         137
во[зс]      122
вы          444
до          153
за          937
и[зс]       276
на          524
над          17
недо         17
ни[зс]       10
о           817
обе[зс]      27
от          393
пере        357
по         1009
под         266
полу         17
пре          68
пред         58
при         411
про         483
ра[зс]      622
с           653
у           466
dtype: int64

In [7]:
final_pref_df.to_csv('prefixes_and_lemmas.tsv', sep='\t', index=False)